# Valuing European-Style Swaptions Across Models

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.products.rates import *
from financepy.finutils import *
from financepy.products.rates.TuringIborSingleCurve import TuringIborSingleCurve
from financepy.market.curves.TuringInterpolator import TuringInterpTypes

####################################################################
# FINANCEPY BETA Version 0.185 - This build:  24 Oct 2020 at 21:27 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Building a Libor Discount Curve

In [4]:
valuationDate = TuringDate(28, 2, 2014)
settlementDate = TuringDate(4, 3, 2014)

We have a vector of dates and zero rates

In [5]:
depoDCCType = TuringDayCountTypes.THIRTY_E_360_ISDA
depos = []

depo = TuringIborDeposit(settlementDate, "1W", 0.0023, depoDCCType); depos.append(depo)
depo = TuringIborDeposit(settlementDate, "1M", 0.0023, depoDCCType); depos.append(depo)
depo = TuringIborDeposit(settlementDate, "3M", 0.0023, depoDCCType); depos.append(depo)
depo = TuringIborDeposit(settlementDate, "6M", 0.0023, depoDCCType); depos.append(depo)

# No convexity correction provided so I omit interest rate futures

swaps = []
swapType = TuringSwapTypes.PAY
fixedDCCType = TuringDayCountTypes.ACT_365F
fixedFreqType = TuringFrequencyTypes.SEMI_ANNUAL

swap = TuringIborSwap(settlementDate, "3Y", swapType, 0.00790, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "4Y", swapType, 0.01200, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "5Y", swapType, 0.01570, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "6Y", swapType, 0.01865, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "7Y", swapType, 0.02160, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "8Y", swapType, 0.02350, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "9Y", swapType, 0.02540, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "10Y", swapType, 0.0273, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "15Y", swapType, 0.0297, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "20Y", swapType, 0.0316, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "25Y", swapType, 0.0335, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "30Y", swapType, 0.0354, fixedFreqType, fixedDCCType); swaps.append(swap)

liborCurve = TuringIborSingleCurve(valuationDate, depos, [], swaps)

Inserting synthetic deposit


In [6]:
exerciseDate = settlementDate.addTenor("5Y")
swapMaturityDate = exerciseDate.addTenor("5Y")
swapFixedCoupon = 0.040852
swapFixedFrequencyType = TuringFrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = TuringDayCountTypes.THIRTY_E_360_ISDA
swapFloatFrequencyType = TuringFrequencyTypes.QUARTERLY
swapFloatDayCountType = TuringDayCountTypes.ACT_360
swapNotional = ONE_MILLION
swapType = TuringSwapTypes.PAY  
calendarType = TuringCalendarTypes.WEEKEND
busDayAdjustType = TuringBusDayAdjustTypes.NONE
dateGenRuleType = TuringDateGenRuleTypes.BACKWARD

In [7]:
swaption = TuringIborSwaption(settlementDate,
                            exerciseDate,
                            swapMaturityDate,
                            swapType,
                            swapFixedCoupon,
                            swapFixedFrequencyType,
                            swapFixedDayCountType, 
                            swapNotional, 
                            swapFloatFrequencyType,
                            swapFloatDayCountType,
                            calendarType, 
                            busDayAdjustType,
                            dateGenRuleType)

## Valuation using Black's Model

In [8]:
model = TuringModelBlack(0.1533)

In [9]:
swaption.value(settlementDate, liborCurve, model)

23163.334872028132

## Valuation using Shifted Black

In [22]:
model = TuringModelBlackShifted(0.1533, 0.008)

In [23]:
swaption.value(settlementDate, liborCurve, model)

27695.460283579072

##  Valuation using SABR

In [12]:
model = TuringModelSABR(0.132, 0.5, 0.5, 0.5)

In [13]:
swaption.value(settlementDate, liborCurve, model)

104583.52188346579

## Valuation using Shifted SABR

In [14]:
model = TuringModelSABRShifted(0.1, 0.2, 0.1, 0.15, -0.005)

In [15]:
swaption.value(settlementDate, liborCurve, model)

163445.66166978356

## Valuation using Hull-White

In [16]:
model = TuringModelRatesHW(0.01, 0.02)

In [17]:
swaption.value(settlementDate, liborCurve, model)

34173.86962564415

## Valuation using Black-Karasinski

In [18]:
model = TuringModelRatesBK(0.2, 0.05)

In [19]:
swaption.value(settlementDate, liborCurve, model)

23474.88575736677

## Internals

In [20]:
print(swaption)

OBJECT TYPE: TuringIborSwaption
SETTLEMENT DATE: TUE 04 MAR 2014
EXERCISE DATE: MON 04 MAR 2019
SWAP TYPE: TuringSwapTypes.PAY
SWAP MATURITY DATE: MON 04 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: TuringFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: TuringDayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: TuringFrequencyTypes.QUARTERLY
FLOAT DAY COUNT: TuringDayCountTypes.ACT_360
PV01: 4.161673891171295
FWD SWAP RATE: 4.086047153125578
FWD DF TO EXPIRY: 0.9237697293451264


We can see that the forward swap rate almost equals the fixed coupon. The underlying swap is close to being ATM forward.

In [21]:
swaption.printSwapFixedLeg()

START DATE: MON 04 MAR 2019
MATURITY DATE: MON 04 MAR 2024
COUPON (%): 4.0852
FIXED LEG FREQUENCY: TuringFrequencyTypes.SEMI_ANNUAL
FIXED LEG DAY COUNT: TuringDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE TUE 04 MAR 2014
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
TUE 04 MAR 2014          -            -   1.00000000            -            -
WED 04 SEP 2019  0.5000000     20426.00   0.90872318     18561.58     18561.58
WED 04 MAR 2020  0.5000000     20426.00   0.89262979     18232.86     36794.44
FRI 04 SEP 2020  0.5000000     20426.00   0.87518398     17876.51     54670.94
THU 04 MAR 2021  0.5000000     20426.00   0.85693776     17503.81     72174.75
SAT 04 SEP 2021  0.5000000     20426.00   0.84102606     17178.80     89353.55
FRI 04 MAR 2022  0.5000000     20426.00   0.82477877     16846.93    106200.48
SUN 04 SEP 2022  0.5000000     20426.00   0.80771756     16498.44    122698.92
SAT 04 MAR 2023  0.5000000     20426.00   0.79039583     16144.63  

Copyright (c) 2020 Dominic O'Kane